In [3]:
import pandas as pd
import os
import copy
from sklearn.cluster import KMeans

In [4]:
filepath = os.getcwd() + '/assignment2/input/TrainTags.csv'
data = pd.read_csv(filepath, encoding='latin1')

In [21]:
data[:5]

,Unnamed: 0,Id,Tag
0,72,2120,sql
1,73,2120,sql-server
2,81,2540,c
3,82,2540,architecture
4,83,2540,data-structures


In [23]:
data.groupby('Id')['Tag'].count().mean()

2.9675617569049431

In [6]:
tagCountSeries = data['Tag'].value_counts()
commonTags = tagCountSeries[:100]
commonTagsSeries = pd.Series(commonTags.index)
commonTags_df = pd.DataFrame({'QnCount':commonTags.values, 'Tag':commonTags.index})

In [7]:
commonTagTotalQnCount = commonTags_df['QnCount'].sum()
def getPopularity(qnCount):
    return qnCount/commonTagTotalQnCount
commonTags_df['Popularity'] = commonTags_df['QnCount'].apply(getPopularity)

In [200]:
# example of looking at one tag row
commonTags_df[commonTags_df['Tag'] == 'sql-server']

,QnCount,Tag,Popularity
21,2621,sql-server,0.010186


In [211]:
# example of looking at tag's popularity
commonTags_df[commonTags_df['Tag'] == 'sql-server']['Popularity']

21    0.010186
Name: Popularity, dtype: float64

In [2]:
len(commonTags_df)

NameError: name 'commonTags_df' is not defined

In [ ]:
# Getting a list of all unique tags in data set
tags = data['Tag'].unique().tolist()
print(tags[:10])
print(tags.index('getter'))

In [40]:
qnids = data['Id'].unique()

185121

In [41]:
# method to one hot encode all tags
def one_hot_encode_tags(tags_list):
    #create an array of 0s for all tags, + 1 for unknown tags
    encoded_tag = [0] * (len(tags) + 1)
    for tag in tags_list:
        index = tags.index(tag) if tag in tags else -1
        encoded_tag[index] = 1
    return encoded_tag

In [192]:
# method to one hot encode most common tags (currently top 100)
def oneHotEncode_common_tags(tags_list):
    return commonTagsSeries.isin(tags_list).apply(int).tolist()

In [ ]:
tags_by_qn = data.groupby('Id')['Tag'].apply(list)
tags_by_qn_df = pd.DataFrame({'qnid':tags_by_qn.index, 'tags':tags_by_qn.values})

In [289]:
tags_by_qn_df[:10]

,qnid,tags,encoded_tags,avg_popularity,max_popularity
0,2120,"[sql, sql-server]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",0.015410,0.020633
1,2540,"[c, architecture, data-structures]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.004440,0.013319
2,2630,"[powershell, cmdlets]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,0.000000
3,2750,"[optimization, setter, getter, verification]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,0.000000
4,2840,"[sql, sql-server-2005, paging]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",0.006878,0.020633
5,4860,"[xml, xml-signature]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.004240,0.008480
6,6110,[database],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.006918,0.006918
7,7190,"[svn, continuous-integration]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,0.000000
8,7540,"[javascript, firefox, memory, cpu]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.017683,0.070732
9,8790,"[ant, build-process]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,0.000000


In [206]:
# One hot encode all tags
# Notes : Takes too long
tags_by_qn_df['encoded_tags'] = tags_by_qn_df['tags'].apply(oneHotEncode_common_tags)

In [261]:
def getQnPopularityScore(tag_list):
    avg_popularity = 0
    max_popularity = 0
    tags_popularity = commonTags_df[commonTags_df['Tag'].isin(tag_list)]['Popularity']
    if len(tags_popularity) > 0:
        avg_popularity = sum(tags_popularity)/len(tag_list)
        max_popularity = max(tags_popularity)
    return avg_popularity,max_popularity

In [ ]:
# Get Popularity score of all Qns based on rating of common tags
tags_by_qn_df['avg_popularity'],tags_by_qn_df['max_popularity'] = \
    zip(*tags_by_qn_df['tags'].map(getQnPopularityScore))

In [290]:
tags_by_qn_df[:10]

,qnid,tags,encoded_tags,avg_popularity,max_popularity
0,2120,"[sql, sql-server]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",0.015410,0.020633
1,2540,"[c, architecture, data-structures]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.004440,0.013319
2,2630,"[powershell, cmdlets]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,0.000000
3,2750,"[optimization, setter, getter, verification]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,0.000000
4,2840,"[sql, sql-server-2005, paging]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",0.006878,0.020633
5,4860,"[xml, xml-signature]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.004240,0.008480
6,6110,[database],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.006918,0.006918
7,7190,"[svn, continuous-integration]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,0.000000
8,7540,"[javascript, firefox, memory, cpu]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.017683,0.070732
9,8790,"[ant, build-process]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,0.000000


In [282]:
# export data
tags_by_qn_df.to_csv('tags_features.csv')

In [ ]:
X = tags_by_qn_df['encoded_tags']
kmeans = KMeans(n_clusters=100, random_state=0).fit(X.tolist())

In [113]:
tags_by_qn_df_small = copy.deepcopy(tags_by_qn_df[:10])
tags_by_qn_df_small['encoded_tags'] = tags_by_qn_df_small['tags'].apply(one_hot_encode_tags)

In [ ]:
# X = []
# for qnid, tag_list in tags_by_qn.iteritems():
#     feature_row = one_hot_encode_tags(tag_list)
#     feature_row.insert(0,qnid)
#     X.append(feature_row)

In [ ]:
def tags_for_qn(qnid):
    qn_tags = data[data['Id'] == qnid]
    qn_tags_list = []
    for t in qn_tags.itertuples():
        qn_tags_list.append(t[3])

    return qn_tags_list

qnids = data['Id'].unique()
print(len(qnids))

In [ ]:
# Each row is [qnid ,  1-hot-encoding of qns tags]
X = [[qn, one_hot_encode_tags(tags_for_qn(qn))] for qn in qnids[:10000]]

In [6]:
data

,Unnamed: 0,Id,Tag
0,72,2120,sql
1,73,2120,sql-server
2,81,2540,c
3,82,2540,architecture
4,83,2540,data-structures
5,88,2630,powershell
6,89,2630,cmdlets
7,90,2750,optimization
8,91,2750,setter
9,92,2750,getter
